In [3]:
procs()

3-element Array{Int64,1}:
 1
 2
 3

In [15]:
addprocs(1)
@everywhere begin
    import Distributions: Uniform
    using DiffEqBase, OrdinaryDiffEq, DiffEqMonteCarlo

    # number of iterations
    n_iter = 10

    # t span and step
    tspan = (0., 100.)
    step = 1.

    # DiffEq 
    pf_func = function (t,u,p,du)
        vel = p[3]*p[2]*u[2]/(p[4]+u[2])
        vabs = p[1]*p[5]*u[1]
        du[1] = -vabs/p[1]
        du[2] = (vabs-vel)/p[3]
    end

    # Defaults
    u0 = [C0=10., C1=0.]
    params = [Default=1., kcat=7.282813exp(-1), Vd=5.275000exp(0), Km=5.257432exp(0), kabs=2.090394exp(0)]

    pf = ParameterizedFunction(pf_func,params)
    prob = ODEProblem(pf,u0,tspan)

    # MonteCarlo
    prob_func1 = function (prob, i)
      prob.f.params[2] = rand(Uniform(6.5, 7.5))
      prob.f.params[3] = rand(Uniform(4.5, 5.5))
      prob.f.params[4] = rand(Uniform(4.5, 5.5))
      prob.f.params[5] = rand(Uniform(1.5, 3.))
      prob
    end
    
    monte_prob = MonteCarloProblem(prob,prob_func=prob_func1)
end

# Solution
@time sim = solve(monte_prob, Tsit5(), saveat=collect(tspan[1]:step:tspan[2]), parallel_type=:threads, num_monte=n_iter)

 15.341303 seconds (2.25 M allocations: 85.252 MB, 0.38% gc time)


In [ ]:
# plot
using Plots
plotly()
plot(sim, xlims = (0.,5.), xlabel="t", ylabel="C0 and C1")

In [2]:
procs()

1-element Array{Int64,1}:
 1